In [105]:
# Importación de librerías:

import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from functools import reduce
import numpy as np
import re

In [106]:
# Definición de urls para descargar datos desde el sitio de la NBA

traditional_per100poss_url = "https://www.nba.com/stats/players/traditional?SeasonType={}&Season={}&PerMode=Per100Possessions"

advanced_per100poss_url = "https://www.nba.com/stats/players/advanced?SeasonType={}&Season={}&PerMode=Per100Possessions"

misc_per100poss_url = "https://www.nba.com/stats/players/misc?SeasonType={}&Season={}&PerMode=Per100Possessions"

scoring_per100poss_url = "https://www.nba.com/stats/players/scoring?SeasonType={}&Season={}&PerMode=Per100Possessions"

defense_per100poss_url = "https://www.nba.com/stats/players/defense?SeasonType={}&Season={}&PerMode=Per100Possessions"

In [107]:
# Definición de urls para descargar datos TRACKING desde el sitio de la NBA

track_drives_url = "https://www.nba.com/stats/players/drives?SeasonType={}&Season={}&PerMode=PerGame"

track_def_impact_url = "https://www.nba.com/stats/players/defensive-impact?SeasonType={}&Season={}&PerMode=PerGame"

track_catchandshoot_url = "https://www.nba.com/stats/players/catch-shoot?SeasonType={}&Season={}&PerMode=PerGame"

track_passing_url = "https://www.nba.com/stats/players/passing?SeasonType={}&Season={}&PerMode=PerGame"

track_pullup_url = "https://www.nba.com/stats/players/pullup?SeasonType={}&Season={}&PerMode=PerGame"

track_reb_url = "https://www.nba.com/stats/players/rebounding?SeasonType={}&Season={}&PerMode=PerGame"

track_offreb_url = "https://www.nba.com/stats/players/offensive-rebounding?SeasonType={}&Season={}&PerMode=PerGame"

track_defreb_url = "https://www.nba.com/stats/players/defensive-rebounding?SeasonType={}&Season={}&PerMode=PerGame"

track_shoot_eff_url = "https://www.nba.com/stats/players/shooting-efficiency?SeasonType={}&Season={}&PerMode=PerGame"

track_speed_url = "https://www.nba.com/stats/players/speed-distance?SeasonType={}&Season={}&PerMode=PerGame"

In [108]:
# Definición de urls para descargar datos OTRAS CATEGORIAS desde el sitio de la NBA

shoot_by_zone_url = "https://www.nba.com/stats/players/shooting?DistanceRange=By+Zone&SeasonType={}&Season={}"


In [109]:
def scrape_nba_historic_data(selected_url, metric_name):
    
    url = selected_url
    web_data = driver.get(url)
    dropdown1 =  Select(driver.find_element(by=By.XPATH, value = '//*[@id="__next"]/div[2]/div[2]/main/div[2]/section/div/div\
                                                [2]/div[1]/div[7]/div/div[3]/div/label/div/select'))
    
    dropdown2 = driver.find_element(by=By.XPATH, value = '//*[@id="__next"]/div[2]/div[2]/main/div[2]/section/div/div[2]/div\
                                        [1]/div[6]/label/div/span')
    time.sleep(5)
    dropdown2.click() # Hacer click en switch para activar los jugadores históricos en lugar de solamente la última temporada
    time.sleep(5)
    dropdown1.select_by_visible_text("All") # Hacer click en menú dropdown para que muestre todas las filas de la tabla
    time.sleep(15) 
    driver.execute_script("window.scrollTo(1,10000)") # Hacer scroll hasta la parte baja de la página para no perder información
    time.sleep(2)

    html_data = driver.page_source
    
    # Escribir información obtenida por el driver en un archivo .html    
    with open("html_data/{}.html".format(metric_name), "w+", encoding="utf-8") as f:
        f.write(html_data)

In [110]:
def scrape_nba_season_data(selected_url, folder_name, metric_name, season, seas_type):
    
    url = selected_url.format(seas_type, season)
    web_data = driver.get(url)
    time.sleep(10)

    dropdown_xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select'
    dropdown1 =  Select(driver.find_element(by=By.XPATH, value = dropdown_xpath))
    dropdown1.select_by_visible_text("All") # Hacer click en menú dropdown para que muestre todas las filas de la tabla
    time.sleep(15) 
    
    driver.execute_script("window.scrollTo(1,10000)") # Hacer scroll hasta la parte baja de la página para no perder información
    time.sleep(2)

    html_data = driver.page_source
    
    # Escribir información obtenida por el driver en un archivo .html    
    with open("html_data/{}/{}.html".format(folder_name, metric_name), "w+", encoding="utf-8") as f:
        f.write(html_data)

In [111]:
def parse_html(folder_name, file_name, id_name, decompose_id, decompose_class):
    with open("html_data/{}/{}.html".format(folder_name, file_name), encoding="utf-8") as f:
        page = f.read()
    soup = BeautifulSoup(page, "html.parser")
    # soup.find(decompose_id, class_=decompose_class).decompose()
    # data_table = soup.find(id=str(id_name))
    data_table = soup.find(class_=str(id_name))
    dataframe = pd.read_html(str(data_table), displayed_only=False)[0]
    
    return dataframe

In [112]:
# Definición de función que transforma dataframe en archivo csv y lo gusrda en la carpeta de dataset

def dataframe_to_csv(dataframe, folder, csv_file_name):
    path = "csv_data/{}/{}.csv".format(folder, csv_file_name)
    dataframe.to_csv(path, index=False)

In [113]:
# Inicialización de driver Chrome para hacer Webscraping

s = Service("D:/Users/user/Desktop/DANI/Programación y DS/Drivers/chromedriver.exe")
driver = webdriver.Chrome(service=s)

In [114]:
current_season = "2023-24"

In [115]:
filename = "regseas_trad_per100_{}".format(current_season)

scrape_nba_season_data(traditional_per100poss_url, folder_name = '2024_season', \
                       metric_name = filename, season = current_season, seas_type = 'Regular+Season')
time.sleep(1)
dataframe = parse_html(folder_name = '2024_season', file_name = filename, id_name="Crom_table__p1iZz", \
                       decompose_id='tr', decompose_class="thead")

In [116]:
dataframe.columns.values

array(['Unnamed: 0', 'Player', 'Team', 'Age', 'GP', 'W', 'L', 'Min',
       'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA',
       'FT%', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'PF',
       'FP', 'DD2', 'TD3', '+/-', 'GP RANK', 'W RANK', 'L RANK',
       'MIN RANK', 'PTS RANK', 'FGM RANK', 'FGA RANK', 'FG% RANK',
       '3PM RANK', '3PA RANK', '3P% RANK', 'FTM RANK', 'FTA RANK',
       'FT% RANK', 'OREB RANK', 'DREB RANK', 'REB RANK', 'AST RANK',
       'TOV RANK', 'STL RANK', 'BLK RANK', 'PF RANK', 'FP RANK',
       'DD2 RANK', 'TD3 RANK', '+/- RANK'], dtype=object)

In [117]:
sel_col_list = ['Player', 'Team', 'Age', 'GP', 'Min', 'FGM', 'FGA',
               '3PM', 'TOV', 'FTM', '+/-', '3PA', ]

In [118]:
dataframe = dataframe[sel_col_list]
dataframe.head(2)

,Player,Team,Age,GP,Min,FGM,FGA,3PM,TOV,FTM,+/-,3PA
0,Drew Peterson,BOS,24,1,51.0,20.0,20.0,20.0,0.0,0.0,40.0,20.0
1,David Duke Jr.,SAS,24,1,53.6,16.7,16.7,16.7,0.0,0.0,-16.7,16.7


In [119]:
dataframe.columns = ['player', 'team', 'age', 'gp', 'min', 'fgm_trad', 'fga_trad',
               '3pm_trad', 'tov_trad', 'ftm_trad', '+/-_trad', '3pa_trad', ]

In [120]:
dataframe_to_csv(dataframe, '2024_season', filename)
print("saved: ", filename)

saved:  regseas_trad_per100_2023-24


In [121]:
filename = "regseas_adv_per100_{}".format(current_season)

scrape_nba_season_data(advanced_per100poss_url, folder_name = '2024_season', \
                       metric_name = filename, season = current_season, seas_type = 'Regular+Season')
time.sleep(1)
dataframe = parse_html(folder_name = '2024_season', file_name = filename, id_name="Crom_table__p1iZz", \
                       decompose_id='tr', decompose_class="thead")

In [122]:
dataframe.columns.values

array(['Unnamed: 0', 'PLAYER', 'TEAM', 'AGE', 'GP', 'W', 'L', 'MIN',
       'OFFRTG', 'DEFRTG', 'NETRTG', 'AST%', 'AST/TO', 'AST\xa0RATIO',
       'OREB%', 'DREB%', 'REB%', 'TO\xa0RATIO', 'EFG%', 'TS%', 'USG%',
       'PACE', 'PIE', 'POSS', 'FGM', 'FGA', 'FGM\xa0PG', 'FGA\xa0PG',
       'FG%', 'GP RANK', 'W RANK', 'L RANK', 'MIN RANK', 'OFFRTG RANK',
       'DEFRTG RANK', 'NETRTG RANK', 'AST% RANK', 'AST/TO RANK',
       'AST\xa0Ratio RANK', 'OREB% RANK', 'DREB% RANK', 'REB% RANK',
       'TO\xa0Ratio RANK', 'eFG% RANK', 'TS% RANK', 'USG% RANK',
       'PACE RANK', 'PIE RANK', 'FGM RANK', 'FGA RANK', 'FGM\xa0PG RANK',
       'FGA\xa0PG RANK', 'FG% RANK'], dtype=object)

In [123]:
sel_col_list = ['PLAYER', 'TEAM', 'PIE', 'POSS', 'TO\xa0RATIO', 'AST/TO']

In [124]:
dataframe = dataframe[sel_col_list]
dataframe.head(2)

,PLAYER,TEAM,PIE,POSS,TO RATIO,AST/TO
0,Buddy Hield,PHI,8.2,4504,8.6,2.29
1,Obi Toppin,IND,10.1,3722,8.4,1.80


In [125]:
dataframe.columns = ['player', 'team', 'pie_adv', 'poss_adv', 
                     'to\xa0ratio_adv', 'ast/to_adv']

In [126]:
dataframe_to_csv(dataframe, '2024_season', filename)
print("saved: ", filename)

saved:  regseas_adv_per100_2023-24


In [127]:
filename = "regseas_misc_per100_{}".format(current_season)

scrape_nba_season_data(misc_per100poss_url, folder_name = '2024_season', \
                       metric_name = filename, season = current_season, seas_type = 'Regular+Season')
time.sleep(1)
dataframe = parse_html(folder_name = '2024_season', file_name = filename, id_name="Crom_table__p1iZz", \
                       decompose_id='tr', decompose_class="thead")

In [128]:
dataframe.columns.values

array(['Unnamed: 0', 'Player', 'TEAM', 'AGE', 'GP', 'W', 'L', 'MIN',
       'PTSOFF\xa0TO', '2ndPTS', 'FBPs', 'PITP', 'Opp\xa0PTSOFF\xa0TO',
       'Opp2nd\xa0PTS', 'OppFBPs', 'OppPITP', 'BLK', 'BLKA', 'PF', 'PFD',
       'GP RANK', 'W RANK', 'L RANK', 'MIN RANK', 'PTSOFF\xa0TO RANK',
       '2ndPTS RANK', 'FBPs RANK', 'PITP RANK',
       'OppPTS\xa0OFF\xa0TO RANK', 'Opp2nd\xa0PTS RANK', 'OppFBPs RANK',
       'OppPITP RANK', 'BLK RANK', 'BLKA RANK', 'PF RANK', 'PFD RANK'],
      dtype=object)

In [129]:
sel_col_list = ['Player', 'TEAM', 'PF', 'BLKA']

In [130]:
dataframe = dataframe[sel_col_list]
dataframe.head(2)

,Player,TEAM,PF,BLKA
0,Buddy Hield,PHI,3.7,0.7
1,Obi Toppin,IND,3.8,0.8


In [131]:
dataframe.columns = ['player', 'team', 'pf_misc', 'blka_misc']

In [132]:
dataframe_to_csv(dataframe, '2024_season', filename)
print("saved: ", filename)

saved:  regseas_misc_per100_2023-24


In [133]:
filename = "regseas_scoring_per100_{}".format(current_season)

scrape_nba_season_data(scoring_per100poss_url, folder_name = '2024_season', \
                       metric_name = filename, season = current_season, seas_type = 'Regular+Season')
time.sleep(1)
dataframe = parse_html(folder_name = '2024_season', file_name = filename, id_name="Crom_table__p1iZz", \
                       decompose_id='tr', decompose_class="thead")

In [134]:
dataframe.columns.values

array(['Unnamed: 0', 'Player', 'TEAM', 'AGE', 'GP', 'W', 'L', 'MIN',
       '%FGA2PT', '%FGA3PT', '%PTS2PT', '%PTS2PT\xa0MR', '%PTS3PT',
       '%PTSFBPs', '%PTSFT', '%PTSOFFTO', '%PTSPITP', '2FGM%AST',
       '2FGM%UAST', '3FGM%AST', '3FGM%UAST', 'FGM%AST', 'FGM%UAST',
       'GP RANK', 'W RANK', 'L RANK', 'MIN RANK', '%FGA2PT RANK',
       '%FGA3PT RANK', '%PTS2PT RANK', '%PTS2PT\xa0MR RANK',
       '%PTS3PT RANK', '%PTSFBPs RANK', '%PTSFT RANK', '%PTSOffTO RANK',
       '%PTSPITP RANK', '2FGM%AST RANK', '2FGM%UAST RANK',
       '3FGM%AST RANK', '3FGM%UAST RANK', 'FGM%AST RANK', 'FGM%UAST RANK'],
      dtype=object)

In [135]:
sel_col_list = ['Player', 'TEAM', 'FGM%AST', '3FGM%AST']

In [136]:
dataframe = dataframe[sel_col_list]
dataframe.head(2)

,Player,TEAM,FGM%AST,3FGM%AST
0,Buddy Hield,PHI,81.8,92.5
1,Obi Toppin,IND,85.3,99.0


In [137]:
dataframe.columns = ['player', 'team', 'fgm%ast_scoring', '3fgm%ast_scoring']

In [138]:
dataframe_to_csv(dataframe, '2024_season', filename)
print("saved: ", filename)

saved:  regseas_scoring_per100_2023-24


In [139]:
filename = "regseas_def_per100_{}".format(current_season)

scrape_nba_season_data(defense_per100poss_url, folder_name = '2024_season', \
                       metric_name = filename, season = current_season, seas_type = 'Regular+Season')
time.sleep(1)
dataframe = parse_html(folder_name = '2024_season', file_name = filename, id_name="Crom_table__p1iZz", \
                       decompose_id='tr', decompose_class="thead")

In [140]:
dataframe.columns.values

array(['Unnamed: 0', 'Player', 'TEAM', 'AGE', 'GP', 'W', 'L', 'MIN',
       'DEF\xa0RTG', 'DREB', 'DREB%', '%DREB', 'STL', 'STL%', 'BLK',
       '%BLK', 'OPP\xa0PTSOFF\xa0TOV', 'OPP\xa0PTS2ND\xa0CHANCE',
       'OPP\xa0PTSFB', 'OPP\xa0PTSPAINT', 'DEFWS', 'GP RANK', 'W RANK',
       'L RANK', 'MIN RANK', 'DEF\xa0RTG RANK', 'DREB RANK', 'DREB% RANK',
       '%DREB RANK', 'STL RANK', 'STL% RANK', 'BLK RANK', '%BLK RANK',
       'OPP\xa0PTSOFF\xa0TOV RANK', 'OPP\xa0PTS2ND\xa0CHANCE RANK',
       'OPP\xa0PTSFB RANK', 'OPP\xa0PTSPAINT RANK', 'DEFWS RANK'],
      dtype=object)

In [141]:
sel_col_list = ['Player', 'TEAM', 'STL%', 'DEFWS', '%DREB',
               'DREB', 'OPP\xa0PTSPAINT', 'OPP\xa0PTSOFF\xa0TOV', 'DEF\xa0RTG',
               'STL', 'DREB%', 'BLK', '%BLK', 'OPP\xa0PTSFB', 'OPP\xa0PTS2ND\xa0CHANCE']

In [142]:
dataframe = dataframe[sel_col_list]
dataframe.head(2)

,Player,TEAM,STL%,DEFWS,%DREB,DREB,OPP PTSPAINT,OPP PTSOFF TOV,DEF RTG,STL,DREB%,BLK,%BLK,OPP PTSFB,OPP PTS2ND CHANCE
0,Justin Jackson,MIN,0.0,1.723,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Andrew Funk,CHI,0.0,1.431,0.0,0.0,0.0,0.0,33.3,0.0,0.0,0.0,0.0,0.0,0.0


In [143]:
dataframe.columns = ['player', 'team', 'stl%_def', 'defws_def', '%dreb_def',
                       'dreb_def', 'opp\xa0ptspaint_def', 'opp\xa0pts\xa0tov_def', 'def\xa0rtg_def',
                       'stl_def', 'dreb%_def', 'blk_def', '%blk_def', 'opp\xa0ptsfb_def', 'opp\xa0pts2nd\xa0chance_def']

In [144]:
dataframe_to_csv(dataframe, '2024_season', filename)
print("saved: ", filename)

saved:  regseas_def_per100_2023-24


In [145]:
filename = "regseas_drives_{}".format(current_season)

scrape_nba_season_data(track_drives_url, folder_name = '2024_season', \
                       metric_name = filename, season = current_season, seas_type = 'Regular+Season')
time.sleep(1)
dataframe = parse_html(folder_name = '2024_season', file_name = filename, id_name="Crom_table__p1iZz", \
                       decompose_id='tr', decompose_class="thead")

In [146]:
dataframe.columns.values

array(['PLAYER', 'TEAM', 'GP', 'W', 'L', 'MIN', 'DRIVES', 'FGM', 'FGA',
       'FG%', 'FTM', 'FTA', 'FT%', 'PTS', 'PTS%', 'PASS', 'PASS%', 'AST',
       'AST%', 'TO', 'TOV%', 'PF', 'PF%'], dtype=object)

In [147]:
sel_col_list = ['PLAYER', 'TEAM', 'PASS%', 'FG%', 'PTS%', 'TOV%',
               'FT%', 'FTM', 'PF%']

In [148]:
dataframe = dataframe[sel_col_list]
dataframe.head(2)

,PLAYER,TEAM,PASS%,FG%,PTS%,TOV%,FT%,FTM,PF%
0,A.J. Lawson,DAL,20.7,22.2,34.5,6.9,33.3,0.1,10.3
1,AJ Green,MIL,63.6,40.0,18.2,9.1,0.0,0.0,0.0


In [149]:
dataframe.columns = ['player', 'team', 'pass%_drives', 'fg%_drives', 'pts%_drives', 'tov%_drives',
                       'ft%_drives', 'ftm_drives', 'pf%_drives']

In [150]:
dataframe_to_csv(dataframe, '2024_season', filename)
print("saved: ", filename)

saved:  regseas_drives_2023-24


In [151]:
filename = "regseas_def_imp_{}".format(current_season)

scrape_nba_season_data(track_def_impact_url, folder_name = '2024_season', \
                       metric_name = filename, season = current_season, seas_type = 'Regular+Season')
time.sleep(1)
dataframe = parse_html(folder_name = '2024_season', file_name = filename, id_name="Crom_table__p1iZz", \
                       decompose_id='tr', decompose_class="thead")

In [152]:
dataframe.columns.values

array(['Player', 'Team', 'GP', 'MIN', 'W', 'L', 'STL', 'BLK', 'DREB',
       'DFGM', 'DFGA', 'DFG%'], dtype=object)

In [153]:
sel_col_list = ['Player', 'Team', 'DFG%', 'DFGA', 'DFGM']

In [154]:
dataframe = dataframe[sel_col_list]
dataframe.head(2)

,Player,Team,DFG%,DFGA,DFGM
0,A.J. Lawson,DAL,72.2,0.5,0.4
1,AJ Green,MIL,76.3,0.7,0.5


In [155]:
dataframe.columns = ['player', 'team', 'dfg%_imp', 'dfga_imp', 'dfgm_imp']

In [156]:
dataframe_to_csv(dataframe, '2024_season', filename)
print("saved: ", filename)

saved:  regseas_def_imp_2023-24


In [157]:
filename = "regseas_cns_{}".format(current_season)

scrape_nba_season_data(track_catchandshoot_url, folder_name = '2024_season', \
                       metric_name = filename, season = current_season, seas_type = 'Regular+Season')
time.sleep(1)
dataframe = parse_html(folder_name = '2024_season', file_name = filename, id_name="Crom_table__p1iZz", \
                       decompose_id='tr', decompose_class="thead")

In [158]:
dataframe.columns.values

array(['PLAYER', 'TEAM', 'GP', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM',
       '3PA', '3P%', 'eFG%'], dtype=object)

In [159]:
sel_col_list = ['PLAYER', 'TEAM', '3P%']

In [160]:
dataframe = dataframe[sel_col_list]
dataframe.head(2)

,PLAYER,TEAM,3P%
0,A.J. Lawson,DAL,30.2
1,AJ Green,MIL,42.2


In [161]:
dataframe.columns = ['player', 'team', '3p%_catch&shoot']

In [162]:
dataframe_to_csv(dataframe, '2024_season', filename)
print("saved: ", filename)

saved:  regseas_cns_2023-24


In [163]:
filename = "regseas_pup_{}".format(current_season)

scrape_nba_season_data(track_pullup_url, folder_name = '2024_season', \
                       metric_name = filename, season = current_season, seas_type = 'Regular+Season')
time.sleep(1)
dataframe = parse_html(folder_name = '2024_season', file_name = filename, id_name="Crom_table__p1iZz", \
                       decompose_id='tr', decompose_class="thead")

In [164]:
dataframe.columns.values

array(['PLAYER', 'TEAM', 'GP', 'W', 'L', 'MIN', 'PTS', 'FGM', 'FGA',
       'FG%', '3PM', '3PA', '3P%', 'eFG%'], dtype=object)

In [165]:
sel_col_list = ['PLAYER', 'TEAM', 'FG%']

In [166]:
dataframe = dataframe[sel_col_list]
dataframe.head(2)

,PLAYER,TEAM,FG%
0,A.J. Lawson,DAL,0.0
1,AJ Green,MIL,36.4


In [167]:
dataframe.columns = ['player', 'team', 'fg%_pullup']

In [168]:
dataframe_to_csv(dataframe, '2024_season', filename)
print("saved: ", filename)

saved:  regseas_pup_2023-24


In [169]:
filename = "regseas_offreb_{}".format(current_season)

scrape_nba_season_data(track_offreb_url, folder_name = '2024_season', \
                       metric_name = filename, season = current_season, seas_type = 'Regular+Season')
time.sleep(1)
dataframe = parse_html(folder_name = '2024_season', file_name = filename, id_name="Crom_table__p1iZz", \
                       decompose_id='tr', decompose_class="thead")

In [170]:
dataframe.columns.values

array(['PLAYER', 'TEAM', 'GP', 'W', 'L', 'MIN', 'OREB', 'ContestedOREB',
       'ContestedOREB%', 'OREBChances', 'OREBChance%',
       'DeferredOREB\xa0Chances', 'AdjustedOREB\xa0Chance%',
       'AVG\xa0OREBDistance'], dtype=object)

In [171]:
sel_col_list = ['PLAYER', 'TEAM', 'OREB']

In [172]:
dataframe = dataframe[sel_col_list]
dataframe.head(2)

,PLAYER,TEAM,OREB
0,A.J. Lawson,DAL,0.6
1,AJ Green,MIL,0.3


In [173]:
dataframe.columns = ['player', 'team', 'oreb_oreb']

In [174]:
dataframe_to_csv(dataframe, '2024_season', filename)
print("saved: ", filename)

saved:  regseas_offreb_2023-24


In [175]:
filename = "regseas_defreb_{}".format(current_season)

scrape_nba_season_data(track_defreb_url, folder_name = '2024_season', \
                       metric_name = filename, season = current_season, seas_type = 'Regular+Season')
time.sleep(1)
dataframe = parse_html(folder_name = '2024_season', file_name = filename, id_name="Crom_table__p1iZz", \
                       decompose_id='tr', decompose_class="thead")

In [176]:
dataframe.columns.values

array(['PLAYER', 'TEAM', 'GP', 'W', 'L', 'MIN', 'DREB', 'ContestedDREB',
       'ContestedDREB%', 'DREBChances', 'DREBChance%',
       'DeferredDREB\xa0Chances', 'AdjustedDREB\xa0Chance%',
       'AVG\xa0DREBDistance'], dtype=object)

In [177]:
sel_col_list = ['PLAYER', 'TEAM', 'AVG\xa0DREBDistance', 'ContestedDREB', 'DREBChances',
               'DREBChance%', 'ContestedDREB%', 'AdjustedDREB\xa0Chance%', 'DREB']

In [178]:
dataframe = dataframe[sel_col_list]
dataframe.head(2)

,PLAYER,TEAM,AVG DREBDistance,ContestedDREB,DREBChances,DREBChance%,ContestedDREB%,AdjustedDREB Chance%,DREB
0,A.J. Lawson,DAL,5.5,0.4,2.4,50.0,32.0,58.1,1.2
1,AJ Green,MIL,8.1,0.2,2.9,58.7,11.1,62.1,1.7


In [179]:
dataframe.columns = ['player', 'team', 'avg\xa0drebdistance_reb', 'contesteddreb_reb', 'drebchances_reb',
                     'drebchance%_reb', 'contesteddreb%_reb', 'adjusteddreb\xa0chance%_reb', 'dreb_reb']

In [180]:
dataframe_to_csv(dataframe, '2024_season', filename)
print("saved: ", filename)

saved:  regseas_defreb_2023-24


In [181]:
filename = "regseas_shootef_{}".format(current_season)

scrape_nba_season_data(track_shoot_eff_url, folder_name = '2024_season', \
                       metric_name = filename, season = current_season, seas_type = 'Regular+Season')
time.sleep(1)
dataframe = parse_html(folder_name = '2024_season', file_name = filename, id_name="Crom_table__p1iZz", \
                       decompose_id='tr', decompose_class="thead")

In [182]:
dataframe.columns.values

array(['PLAYER', 'TEAM', 'GP', 'W', 'L', 'MIN', 'PTS', 'DrivePTS',
       'DriveFG%', 'C&SPTS', 'C&SFG%', 'Pull\xa0UpPTS', 'Pull\xa0UpFG%',
       'PaintTouch\xa0PTS', 'PaintTouch\xa0FG%', 'PostTouch\xa0PTS',
       'PostTouch\xa0FG%', 'ElbowTouch\xa0PTS', 'ElbowTouch\xa0FG%',
       'eFG%'], dtype=object)

In [183]:
sel_col_list = ['PLAYER', 'TEAM', 'PaintTouch\xa0PTS', 'Pull\xa0UpFG%']

In [184]:
dataframe = dataframe[sel_col_list]
dataframe.head(2)

,PLAYER,TEAM,PaintTouch PTS,Pull UpFG%
0,A.J. Lawson,DAL,0.6,0.0
1,AJ Green,MIL,0.1,36.4


In [185]:
dataframe.columns = ['player', 'team', 'painttouch\xa0pts_shooting_eff', 'pull\xa0upfg%_shooting_eff']

In [186]:
dataframe_to_csv(dataframe, '2024_season', filename)
print("saved: ", filename)

saved:  regseas_shootef_2023-24


In [187]:
filename = "regseas_speed_{}".format(current_season)

scrape_nba_season_data(track_speed_url, folder_name = '2024_season', \
                       metric_name = filename, season = current_season, seas_type = 'Regular+Season')
time.sleep(1)
dataframe = parse_html(folder_name = '2024_season', file_name = filename, id_name="Crom_table__p1iZz", \
                       decompose_id='tr', decompose_class="thead")

In [188]:
dataframe.columns.values

array(['PLAYER', 'TEAM', 'GP', 'W', 'L', 'MIN', 'Dist.\xa0Feet',
       'Dist.\xa0Miles', 'Dist.\xa0Miles\xa0Off', 'Dist.\xa0Miles\xa0Def',
       'Avg\xa0Speed', 'Avg\xa0Speed\xa0Off', 'Avg\xa0Speed\xa0Def'],
      dtype=object)

In [189]:
sel_col_list = ['PLAYER', 'TEAM', 'Dist.\xa0Miles\xa0Def', 'Avg\xa0Speed\xa0Off']

In [190]:
dataframe = dataframe[sel_col_list]
dataframe.head(2)

,PLAYER,TEAM,Dist. Miles Def,Avg Speed Off
0,A.J. Lawson,DAL,0.28,4.75
1,AJ Green,MIL,0.40,4.72


In [191]:
dataframe.columns = ['player', 'team', 'dist.\xa0miles\xa0def_speed', 'avg\xa0speed\xa0off_speed']

In [192]:
dataframe_to_csv(dataframe, '2024_season', filename)
print("saved: ", filename)

saved:  regseas_speed_2023-24


In [193]:
filename = "regseas_shootzone_pergame_{}".format(current_season)
scrape_nba_season_data(shoot_by_zone_url, folder_name = '2024_season', \
                       metric_name = filename, season = current_season, seas_type = 'Regular+Season')
time.sleep(1)
dataframe = parse_html(folder_name = '2024_season', file_name = filename, id_name="Crom_table__p1iZz", \
                       decompose_id='tr', decompose_class="thead")

dataframe.columns = dataframe.columns.droplevel() 

shzone_columns = ['Player', 'Team', 'Age', 'RA_FGM', 'RA_FGA', 'RA_FG%', 'PAINT_FGM', 'PAINT_FGA', 'PAINT_FG%',\
               'MR_FGM', 'MR_FGA', 'MR_FG%', 'LC3_FGM', 'LC3_FGA', 'LC3_FG%', 'RC3_FGM', 'RC3_FGA', 'RC3_FG%', \
              'C3_FGA', 'C3_FGA', 'C3_FGA', 'AB3_FGM', 'AB3_FGA', 'AB3_FG%']

drop_columns = ['Unnamed: 24_level_1', 'Unnamed: 25_level_1', 'Unnamed: 26_level_1']

dataframe = dataframe.drop(drop_columns, axis=1)
dataframe.columns = shzone_columns



In [194]:
dataframe.columns.values

array(['Player', 'Team', 'Age', 'RA_FGM', 'RA_FGA', 'RA_FG%', 'PAINT_FGM',
       'PAINT_FGA', 'PAINT_FG%', 'MR_FGM', 'MR_FGA', 'MR_FG%', 'LC3_FGM',
       'LC3_FGA', 'LC3_FG%', 'RC3_FGM', 'RC3_FGA', 'RC3_FG%', 'C3_FGA',
       'C3_FGA', 'C3_FGA', 'AB3_FGM', 'AB3_FGA', 'AB3_FG%'], dtype=object)

In [195]:
sel_col_list = ['Player', 'Team', 'MR_FG%']

In [196]:
dataframe = dataframe[sel_col_list]
dataframe.head(2)

,Player,Team,MR_FG%
0,A.J. Lawson,DAL,50.0
1,AJ Green,MIL,53.3


In [197]:
dataframe.columns = ['player', 'team', 'mr_fg%_shootingzone']

In [198]:
dataframe_to_csv(dataframe, 'shooting_by_zone', filename)
print("saved: ", filename)

saved:  regseas_shootzone_pergame_2023-24
